<a href="https://colab.research.google.com/github/Umercia/my_kindle_highlight_export/blob/main/kindle_highlight_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Objectif: format hightlighted texte from kindle ebook reader. 
raw highlight are register in the kindle memory in:
 file called 'Kindle/documents/My_clipping.txt' 

# 1.0 Set-up



In [1]:
from datetime import datetime
import pandas as pd
import re
from tqdm import tqdm
from pathlib import Path
import regex as re
import numpy as np

In [78]:
# variables
ROOT = Path('/content/drive/MyDrive/Colab_Notebooks/kindle_export_notes')
INPUT_DATA_PATH = ROOT / 'data' / 'input'
OUTPUT_DATA_PATH = ROOT / 'data' / 'output' / 'raw'

In [3]:
# mount drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [6]:
# functions

def get_page_nb(txt):

    line_idx = re.search(r"\d+", txt)
    return int(line_idx.group())

def get_location(txt):

    location = re.search(r"\d+-\d+|\d+", txt)
    return location.group()

def get_date(txt):

    date = re.search(r'\d+.*', txt).group()
    return date

months_fr_2_eng = {'janvier' : 'January',
                   'février': 'february',
                   'mars': 'may',
                   'avril': 'april',
                   'mai': 'may',
                   'juin': 'june',
                   'juillet': 'jully',
                   'août': 'august',
                   'septembre': 'september',
                   'octobre': 'october',
                   'novembre': 'november',
                   'décembre': 'december'}

# 2.0 read and process data

In [99]:
# read 'My Clipping.txt'
with open(INPUT_DATA_PATH / 'My Clippings.txt', 'r', encoding="utf8") as f:
    file_txt_content = f.read()

# parse to first lvl: notes
note_delimitation_pattern = r'(.+?)\n={10,}\n'
notes = re.findall(note_delimitation_pattern, file_txt_content, flags=re.DOTALL)

In [103]:
# parse each note information

df = pd.DataFrame(columns=['book_title', 'page', 'location', 'date', 'text'])

extract_elements_pattern =         r'^(.*)\n(.*)\|(.*)\|(.*)\n*([\s\S]*)$'
extract_elements_no_page_pattern = r'^(.*)\n(.*)\|(.*)\n*([\s\S]*)$'

for note in tqdm(notes):  

    try:
        elements = re.findall(extract_elements_pattern, note)[0]
    
        title = elements[0]
        page = get_page_nb(elements[1])
        location = get_location(elements[2])
        date = get_date(elements[3])
        highlighted = elements[4]
    
    except:   # handle missing element due to missing page information
        elements = re.findall(extract_elements_no_page_pattern, note)[0]

        title = elements[0]
        page = None
        location = get_location(elements[1])
        date = get_date(elements[2])
        highlighted = elements[3]

    df.loc[len(df)] = [title, page, location, date, highlighted]


100%|██████████| 4706/4706 [00:12<00:00, 374.30it/s]


In [104]:
# cleaning and processing

# book titles, rename some books for shorter name
book_title_mapping = { 
                        'How to Take Smart Notes: One Simple Technique to Boost Writing, Learning and Thinking – for Students, Academics and Nonfiction Book Writers (Sönke Ahrens)': 'How to Take Smart Notes',
                        'Le jardinier-maraîcher - 2ème édition (Guides pratiques) (French Edition) (Fortier, Jean-Martin)':'Le jardinier-maraîcher',
                        'Le potager du paresseux - Produire en abondance des légumes plus que bio, sans compost, sans travail du sol, sans buttes - nouvelle édition augmentée et illustrée (French Edition) (HELMSTETTER, Didier)':'Le potager du paresseux',   
                        "\ufeffPhilippe de Villiers (Le moment est venu de dire ce que j'ai vu-ALBIN MICHEL (2015))":"Le moment est venu de dire ce que j'ai vu",
                      }

df['book_title'] = (df['book_title'].replace(book_title_mapping)
                                    .str.split('(', expand=True)[0]
                                    .str.strip()
                                    )

# formating & highlight locations parsing
df[['loc_start', 'loc_end']] = (df['location'].str.split('-', expand=True)
                                              .astype(float))

df['text'] = df['text'].replace('','XXX')

mask = df['text'] == 'XXX'
df.loc[mask,'loc_end'] = df.loc[mask,'loc_start']
df['loc_start_i+1'] = df['loc_start'].shift(-1)
df = (df.sort_values(by=['book_title', 'loc_start', 'loc_end'])
        .reset_index(drop=True)
        .drop_duplicates())

# handle title notes, marked with a "note" refering to title level (1, 2, 3)
df['is_title'] = 0
for idx in df.index[:-1]:
    if (df.loc[idx,'loc_end'] >=  df.loc[idx+1,'loc_start']) & (df.loc[idx+1,'text'] in ['1', '2', '3']):
        df.loc[idx,'is_title'] = int(df.loc[idx+1,'text'])

In [105]:
df.sample(10)

,book_title,page,location,date,text,loc_start,loc_end,loc_start_i+1,is_title
791,Atomic Habits,181,2339,30 March 2023 19:07:37,XXX,2339.0,2339.0,2340.0,1
4358,Toxic Data,78,752,17 January 2023 09:08:01,1,752.0,NaN,774.0,0
312,"3 jours de jeûne, c'est parti !",114,1312-1312,13 July 2022 18:21:39,<You have reached the clipping limit for this...,1312.0,1312.0,1312.0,0
4533,Toxic Data,336,3296,2 March 2023 20:34:24,2,3296.0,NaN,3322.0,0
4690,﻿One Small Step Can Change Your Life: The Kaiz...,97,1487-1487,20 mai 2019 07:45:14,The reward should be appropriate to the person.,1487.0,1487.0,1497.0,0
3986,The Big Book of Dashboards,377,5773-5776,15 July 2019 07:35:47,The y-axis in Figure 31.16 does not start at z...,5773.0,5776.0,5785.0,0
1954,Le jardinier-maraîcher,None,1730-1731,20 June 2021 19:28:06,La subdivision de nos jardins en 10 parcelles ...,1730.0,1731.0,1731.0,0
1302,Effondrement,328,5023,5 March 2020 19:59:30,1,5023.0,NaN,86.0,0
3346,L’horloge de la nature,135,1799,28 September 2021 20:34:59,2,1799.0,NaN,1799.0,0
407,AI Superpowers,59,896,2 August 2019 18:10:29,1,896.0,NaN,1144.0,0


# 3.0 Export Highlights

In [89]:
# export hihlights to files (one per book)
book_titles = df['book_title'].unique()

for book_title in book_titles:
    indent_spaces = 0
    with open(OUTPUT_DATA_PATH / f'{book_title}.txt', "w") as f:

        for row in df.query('(book_title == @book_title) & (loc_end > 0)').drop_duplicates(subset=['loc_start'], keep='last').itertuples():
            if row.is_title > 0:
                f.write('\n')
                indent_spaces = int(row.is_title)
                f.write('    '*(indent_spaces) + row.text.upper())
                f.write("\n")
                f.write('    '*(indent_spaces) + '-'* (len(row.text)))
                f.write("\n")
                
            else:
                f.write('    '*(indent_spaces+1) + '-' + row.text)
                f.write("\n")